In [1]:
import numpy as np
import pandas as pd 
import requests 
import xlsxwriter 
import math

In [2]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [3]:
df = pd.read_csv('S&P500.csv')
df.head()

,Symbol
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [4]:
Companies = {}  # stores company names along with mkt cap and latest price
Companies['Name'] = []
Companies['CMP'] = []
Companies['Market Cap'] = []

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
batches  = list(chunks(df['Symbol'], 100)) # 

for i in range(len(batches)):
    batch = (','.join(batches[i]))
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for company in batch.split(','):
        Companies['Name'].append(company)
        Companies['CMP'].append(data[company]['quote']['latestPrice'])
        Companies['Market Cap'].append(data[company]['quote']['marketCap'])
        

df = pd.DataFrame(Companies)
df

,Name,CMP,Market Cap
0,A,123.08,3.730272e+10
1,AAL,14.00,9.132074e+09
2,AAP,187.50,1.165293e+10
3,AAPL,146.54,2.393767e+12
4,ABBV,154.91,2.744132e+11
...,...,...,...
497,YUM,119.22,3.413754e+10
498,ZBH,113.89,2.342755e+10
499,ZBRA,324.03,1.689479e+10
500,ZION,55.22,8.420706e+09


In [5]:
df = pd.DataFrame(Companies)
df

,Name,CMP,Market Cap
0,A,123.08,3.730272e+10
1,AAL,14.00,9.132074e+09
2,AAP,187.50,1.165293e+10
3,AAPL,146.54,2.393767e+12
4,ABBV,154.91,2.744132e+11
...,...,...,...
497,YUM,119.22,3.413754e+10
498,ZBH,113.89,2.342755e+10
499,ZBRA,324.03,1.689479e+10
500,ZION,55.22,8.420706e+09


In [6]:
portfolio_size = 100000

In [7]:
# Shares for each company
amount = portfolio_size/len(df['Name'])

pd.set_option('mode.chained_assignment', None)
df['Shares to buy'] = 0
for i in range(len(df['Name'])):
    df['Shares to buy'][i] = math.floor(amount/df['CMP'][i])
    
df

,Name,CMP,Market Cap,Shares to buy
0,A,123.08,3.730272e+10,1
1,AAL,14.00,9.132074e+09,14
2,AAP,187.50,1.165293e+10,1
3,AAPL,146.54,2.393767e+12,1
4,ABBV,154.91,2.744132e+11,1
...,...,...,...,...
497,YUM,119.22,3.413754e+10,1
498,ZBH,113.89,2.342755e+10,1
499,ZBRA,324.03,1.689479e+10,0
500,ZION,55.22,8.420706e+09,3


In [8]:
df.to_csv("shares.csv")